In [1]:
from tool import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def accumulate_sp_history(total_sp, new_sp, idx_cols, ref_week):
    deleted_wk = []
    for wk in get_weeklist(total_sp):
        if wk not in get_weeklist(new_sp):
            deleted_wk.append(wk)
    org_idx = total_sp.reset_index().set_index(idx_cols).index
    new_idx = new_sp.reset_index().set_index(idx_cols).index
    intersecting_idx = org_idx.intersection(new_idx)
    diff_idx_org = org_idx.difference(intersecting_idx)
    
    find_most_recent_week = total_sp.reset_index()[['REF']+idx_cols].groupby(idx_cols)['REF'].max()
    idx_frame1 = find_most_recent_week[diff_idx_org].reset_index()[['REF'] + idx_cols]
    m_idx1 = pd.MultiIndex.from_frame(idx_frame1)
    add_df = total_sp.loc[m_idx1, :get_weekname_from(ref_week, -2)]
    for wk in find_most_recent_week[diff_idx_org].unique():
        if wk < get_weekname_from(ref_week, -1):
            add_df.loc[wk, wk:] = 0
            
    add_df = add_df[add_df.sum(axis=1) > 0]
    
    if len(deleted_wk) > 0:
        idx_frame2 = find_most_recent_week[intersecting_idx].reset_index()[['REF'] + idx_cols]
        idx_frame3 = idx_frame2.copy()
        idx_frame3['REF'] = ref_week
        m_idx2 = pd.MultiIndex.from_frame(idx_frame2)
        m_idx3 = pd.MultiIndex.from_frame(idx_frame3)
        new_sp.loc[m_idx3, deleted_wk] = total_sp.loc[m_idx2, deleted_wk].values
        
    new_sp = pd.concat([add_df, new_sp]).reset_index()
    new_sp['REF'] = ref_week
    new_sp = new_sp.set_index(['REF'] + idx_cols)
    
    return pd.concat([total_sp, new_sp])

# odm release 한 엑셀 파일의 경우
def read_sp_from_excel(filepath, tab_name, vendor, skip_value, ref_week, idx_cols, ver):
    sp = pd.read_excel(filepath, sheet_name=tab_name, skiprows=skip_value, keep_default_na=False, na_values=['', 'nan'])
    trim_column_name(sp)
    sp = sp[sp['From Site'].str.contains(vendor.upper()) & sp['Category'].str.contains(ver)]
    sp = sp[idx_cols + get_weeklist(sp)]
    sp.insert(0, 'REF', ref_week)
    return sp.groupby(['REF']+idx_cols).sum(numeric_only=True)

In [9]:
# gscp에서 다운받은 엑셀 파일의 경우
def read_sp_from_excel_gscp(filepath, vendor, ref_week, idx_cols, ver):
    sp = pd.read_excel(filepath, keep_default_na=False, na_values=['', 'nan'])
    trim_column_name(sp)
    sp = sp[sp['From Site'].str.contains(vendor.upper()) & sp['Category'].str.contains(ver)]
    sp = sp[idx_cols + get_weeklist(sp)]
    sp.insert(0, 'REF', ref_week)
    return sp.groupby(['REF']+idx_cols).sum(numeric_only=True)

### 변수 설정

In [7]:
ver = 'Final'
vendor = 'Quanta'
# idx_cols = ['From Site', 'To Site', 'Region', 'Model.Suffix'] # 인덱스로 사용할 컬럼들
idx_cols = ['From Site', 'To Site', 'Region', 'Mapping Model.Suffix'] # 인덱스로 사용할 컬럼들
path = 'D:/SP현황/Quanta_2019_2023/' # 엑셀 파일의 경로명
os.chdir('D:/SP현황/Quanta_2019_2023/') # 엑셀파일이 있는 폴더로 이동

In [4]:
ref_week = '2018-12-31(W01)'

### 누적할 SP의 초기값 설정

In [5]:
filepath = path + os.listdir()[0]
filepath

'D:/SP현황/Quanta_2019_2023/2019_W01.xls'

In [4]:
# ODM release 엑셀에서 작업할 경우
try:
    tab_name = 'SP_' + filepath.split('.')[0][-5:-1]
    total_sp = read_sp_from_excel(filepath, tab_name, vendor, 5, '2021-05-10(W19)', idx_cols, ver)
except:
    tab_name = 'W' + filepath.split('.')[0][-5:-1]
    total_sp = read_sp_from_excel(filepath, tab_name, vendor, 5, '2021-05-10(W19)', idx_cols, ver)
ref_week = '2021-05-17(W20)'
for filename in tqdm(os.listdir()[1:]): # 엑셀 파일이 들어있는 폴더의 파일들 불러와 반복
    filepath = path + filename
    try:
        tab_name = 'SP_' + filepath.split('.')[0][-5:-1]
        sp = read_sp_from_excel(filepath, tab_name, vendor, 5, ref_week, idx_cols, ver)
    except:
        tab_name = 'W' + filepath.split('.')[0][-5:-1]
        sp = read_sp_from_excel(filepath, tab_name, vendor, 5, ref_week, idx_cols, ver)
    total_sp = accumulate_sp_history(total_sp, sp, idx_cols, ref_week)
    ref_week = get_weekname_from(ref_week, 1)

  1%|▋                                                                                 | 1/110 [00:00<00:32,  3.38it/s]C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_9576\3694933484.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  add_df.loc[wk, wk:] = 0
  2%|█▍                                                                                | 2/110 [00:00<00:33,  3.25it/s]C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_9576\3694933484.py:22: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  add_df.loc[wk, wk:] = 0
100%|███████████████████

In [13]:
# GSCP에서 다운받은 엑셀에서 작업할 경우
total_sp = read_sp_from_excel_gscp(filepath, vendor, ref_week, idx_cols, ver)
ref_week = get_weekname_from(ref_week, 1)
for filename in tqdm(os.listdir()[1:]): # 엑셀 파일이 들어있는 폴더의 파일들을 불러와 반복
    filepath = path + filename
    sp = read_sp_from_excel_gscp(filepath, vendor, ref_week, idx_cols, ver)
    total_sp = accumulate_sp_history(total_sp, sp, idx_cols, ref_week)
    ref_week = get_weekname_from(ref_week, 1)

100%|████████████████████████████████████████████████████████████████████████████████| 235/235 [02:27<00:00,  1.59it/s]


In [14]:
df = total_sp.reset_index()
df.insert(4, 'Series', df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model))
df = df.loc[(df.set_index(['REF', 'Series']+idx_cols).fillna(0).sum(axis=1) != 0).values].fillna(0).reset_index(drop=True) # 전체구간 선적값이 0 인 행은 삭제
df

,REF,From Site,To Site,Region,Series,Mapping Model.Suffix,2018-12-31(W01),2019-01-07(W02),2019-01-14(W03),2019-01-21(W04),...,2023-10-23(W43),2023-10-30(W44),2023-11-06(W45),2023-11-13(W46),2023-11-20(W47),2023-11-27(W48),2023-12-04(W49),2023-12-11(W50),2023-12-18(W51),2023-12-25(W52)
0,2019-01-07(W02),OS_QUANTA_CN_P,EKHQ,KOREA,15G880,15G880-S.AD70K,100.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-01-07(W02),OS_QUANTA_CN_P,EKHQ,KOREA,15G880,15G880-S.AD76K,89.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-01-07(W02),OS_QUANTA_CN_P,EKHQ,KOREA,15G880,15GD880-X.AX76K,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-01-07(W02),OS_QUANTA_CN_P,EKHQ,KOREA,15U480,15U480-G.AA56K,39.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-01-07(W02),OS_QUANTA_CN_P,EKHQ,KOREA,15U480,15U480-G.AA5IK,50.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78889,2023-07-10(W28),OS_QUANTA_CN_P,ENUS_V,NA,16T90Q,16T90Q-K.APB7U1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78890,2023-07-10(W28),OS_QUANTA_CN_P,ENUS_V,NA,16T90R,16T90R-K.AAC7U1,0.0,0.0,0.0,0.0,...,20.0,20.0,0.0,20.0,40.0,50.0,70.0,60.0,0.0,0.0
78891,2023-07-10(W28),OS_QUANTA_CN_P,ENUS_V,NA,16T90R,16T90R-K.ADB9U1,0.0,0.0,0.0,0.0,...,500.0,330.0,150.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0
78892,2023-07-10(W28),OS_QUANTA_CN_P,ENUS_V,NA,16T90R,16T90R-K.ADS9U1,0.0,0.0,0.0,0.0,...,50.0,50.0,50.0,50.0,50.0,0.0,40.0,0.0,0.0,0.0


In [16]:
pdr = get_pdr()

In [17]:
df = pd.merge(df, pdr[['Model.Suffix', 'USB DVD']], left_on='Mapping Model.Suffix', right_on='Model.Suffix', how='left').drop('Model.Suffix', axis=1)

In [39]:
df.to_clipboard()

In [18]:
df = df[df['USB DVD'].notnull()]

In [19]:
df = df[~(df['USB DVD'] == 'WITH OUT')]

In [34]:
df1 = monthly_sum(df.groupby('REF').sum(numeric_only=True), 'REF')
df1

,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
REF,,,,,,,,,,,,,,,,,,,,,
2020-05-25(W22),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-01(W23),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-08(W24),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-15(W25),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-22(W26),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-12(W24),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,190.0,145.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-19(W25),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,190.0,145.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-06-26(W26),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,190.0,145.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df1 = df1.loc[:, '2020-06':]

In [31]:
df1 = df1.stack().reset_index()
df1 = df1.rename(columns={'level_1':'Month', 0:'QTY'})
df1

,REF,Month,QTY
0,2020-05-25(W22),2020-06,180.0
1,2020-05-25(W22),2020-07,0.0
2,2020-05-25(W22),2020-08,1050.0
3,2020-05-25(W22),2020-09,0.0
4,2020-05-25(W22),2020-10,0.0
...,...,...,...
7047,2023-07-10(W28),2023-08,0.0
7048,2023-07-10(W28),2023-09,0.0
7049,2023-07-10(W28),2023-10,0.0
7050,2023-07-10(W28),2023-11,0.0


In [33]:
px.bar(df1, x='REF', y='QTY', color='Month')

In [59]:
with pd.ExcelWriter(f'D:/Python result/Pegatron_sp_release_history.xlsx') as writer:
    df.to_excel(writer, sheet_name='Raw Data')
    monthly_sum(df.groupby(['Series', 'REF']).sum(numeric_only=True), ['Series', 'REF']).to_excel(writer, sheet_name='Summary', index=True)

In [62]:
for sr in df1['Series'].unique():
    fig = px.line(df1[df1['Series'] == sr].groupby('REF').sum(numeric_only=True))
    fig.write_image(f'D:/figure/{sr}.png')
    print(sr)

13U50N
13U70P
13U70Q
14T30Q
14T90N
14U30P
14U30Q
14U390
17U70N
17U70P
17U70Q


In [65]:
from openpyxl.drawing.image import Image
from openpyxl import Workbook
from openpyxl import load_workbook

fn = get_filename()
wb = load_workbook(fn)

for name in df1['Series'].unique():
    wb.create_sheet(name)
    sh = wb[f'{name}']
    add_image = Image(f'D:/figure/{name}.png')
    sh.add_image(add_image, 'A1')
    
wb.save(fn)

In [41]:
df1 = monthly_sum(df.groupby(['Series', 'REF']).sum(numeric_only=True), ['Series', 'REF']).stack().reset_index()

fig = make_subplots(rows=(len(df1['Series'].unique()) + 3) // 4, cols=4, subplot_titles=df1['Series'].unique(), shared_xaxes=True, horizontal_spacing=0.03, vertical_spacing=0.05)
i = 0
row_num = 1
col_num = 1
for sr in df1['Series'].unique():
    df2 = df1[df1['Series'] == sr]
    df2 = df2.groupby('REF').sum(numeric_only=True).reset_index()
    fig.add_trace(go.Line(x=df2['REF'], y=df2[0], name=sr), row=row_num, col=col_num)
    i += 1
    row_num = i // 4 + 1
    col_num = i % 4 + 1
fig.update_layout(height=1000, showlegend=True)
fig.update_xaxes(tickangle=-65)


C:\Users\paul76.lee\AppData\Local\miniconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


C:\Users\paul76.lee\AppData\Local\miniconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


C:\Users\paul76.lee\AppData\Local\miniconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


C:\Users\paul76.lee\AppData\Local\miniconda3\lib\site-packages\p

In [6]:
df['Sum'] = df.sum(axis=1, numeric_only=True)

In [7]:
df.groupby(['Series', 'REF'])['Sum'].sum().reset_index()

,Series,REF,Sum
0,13U50N,2021-05-10(W19),270.0
1,13U50N,2021-05-17(W20),270.0
2,13U50N,2021-05-24(W21),270.0
3,13U50N,2021-05-31(W22),270.0
4,13U50N,2021-06-07(W23),270.0
...,...,...,...
1100,17U70Q,2023-05-22(W21),10605.0
1101,17U70Q,2023-05-29(W22),10595.0
1102,17U70Q,2023-06-05(W23),10595.0
1103,17U70Q,2023-06-12(W24),10595.0


In [294]:
df = df.groupby(['Series', 'REF'])['Sum'].sum().reset_index()
df[df['Series'] == '13U70Q']

,Series,REF,Sum
220,13U70Q,2021-05-24(W21),1050.0
221,13U70Q,2021-05-31(W22),4813.0
222,13U70Q,2021-06-07(W23),5011.0
223,13U70Q,2021-06-14(W24),5572.0
224,13U70Q,2021-06-21(W25),5572.0
...,...,...,...
323,13U70Q,2023-05-15(W20),15739.0
324,13U70Q,2023-05-22(W21),15739.0
325,13U70Q,2023-05-29(W22),15739.0
326,13U70Q,2023-06-05(W23),15739.0


In [242]:
df_plot = monthly_sum(df.groupby(['Series', 'REF']).sum(numeric_only=True).reset_index(), ['Series', 'REF']).stack().reset_index()
df_plot.rename(columns={'level_2':'Month', 0:'QTY'}, inplace=True)
df_plot

,Series,REF,Month,QTY
0,13U50N,2021-05-10(W19),2021-04,270.0
1,13U50N,2021-05-10(W19),2021-05,0.0
2,13U50N,2021-05-10(W19),2021-06,0.0
3,13U50N,2021-05-10(W19),2021-07,0.0
4,13U50N,2021-05-10(W19),2021-08,0.0
...,...,...,...,...
36097,17U70Q,2023-06-12(W24),2023-08,0.0
36098,17U70Q,2023-06-12(W24),2023-09,0.0
36099,17U70Q,2023-06-12(W24),2023-10,0.0
36100,17U70Q,2023-06-12(W24),2023-11,0.0


In [257]:
df_plot = df_plot.groupby(['Series', 'REF'])['QTY'].sum(numeric_only=True).reset_index()
df_plot

,Series,REF,QTY
0,13U50N,2021-05-10(W19),270.0
1,13U50N,2021-05-17(W20),270.0
2,13U50N,2021-05-24(W21),270.0
3,13U50N,2021-05-31(W22),270.0
4,13U50N,2021-06-07(W23),270.0
...,...,...,...
1089,17U70Q,2023-05-15(W20),10595.0
1090,17U70Q,2023-05-22(W21),10605.0
1091,17U70Q,2023-05-29(W22),10595.0
1092,17U70Q,2023-06-05(W23),10595.0


In [268]:
px.line(df_plot[df_plot['Series'] == '14U390'], x='REF', y='QTY')

In [271]:
df_plot[df_plot['Series'] == '14T30Q']

,Series,REF,QTY
328,14T30Q,2022-03-28(W13),80000.0
329,14T30Q,2022-04-04(W14),80000.0
330,14T30Q,2022-04-11(W15),80000.0
331,14T30Q,2022-04-18(W16),68800.0
332,14T30Q,2022-04-25(W17),68800.0
...,...,...,...
387,14T30Q,2023-05-15(W20),11730.0
388,14T30Q,2023-05-22(W21),11730.0
389,14T30Q,2023-05-29(W22),11730.0
390,14T30Q,2023-06-05(W23),11910.0


In [266]:
fig = make_subplots(rows=(len(df_plot['Series'].unique()) + 3) // 4, cols=4, subplot_titles=df_plot['Series'].unique(), shared_xaxes=True, horizontal_spacing=0.03, vertical_spacing=0.05)
i = 0
row_num = 1
col_num = 1
for sr in df_plot['Series'].unique():
    df_sr = df_plot[df_plot['Series'] == sr]
    fig.add_trace(go.Line(x=df_sr['REF'], y=df_sr['QTY'], name=sr), row=row_num, col=col_num)
    i += 1
    row_num = i // 4 + 1
    col_num = i % 4 + 1
fig.update_layout(height=1000, showlegend=True)
fig.update_xaxes(tickangle=-65)

In [240]:
px.line(df_plot)

,Series,REF,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
0,13U50N,2021-05-10(W19),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13U50N,2021-05-17(W20),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13U50N,2021-05-24(W21),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13U50N,2021-05-31(W22),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13U50N,2021-06-07(W23),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1089,17U70Q,2023-05-15(W20),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2863.0,540.0,20.0,630.0,0.0,0.0,0.0,0.0,0.0,0.0
1090,17U70Q,2023-05-22(W21),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2863.0,540.0,0.0,650.0,0.0,0.0,0.0,0.0,10.0,0.0
1091,17U70Q,2023-05-29(W22),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2863.0,540.0,0.0,650.0,0.0,0.0,0.0,0.0,0.0,0.0
1092,17U70Q,2023-06-05(W23),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2863.0,540.0,0.0,650.0,0.0,0.0,0.0,0.0,0.0,0.0


In [238]:
for model in df_plot.index.get_level_values(0).unique():
    df_plot.loc[model]

13U50N
13U70P
13U70Q
14T30Q
14T90N
14U30P
14U30Q
14U390
17U70N
17U70P
17U70Q


In [229]:
df_plot = df.groupby(['Series', 'REF']).sum(numeric_only=True).sum(axis=1).loc['13U50N']
df_plot.name = '13U50N'

In [230]:
if df_plot[df_plot != df_plot[-1]].shape[0] == 0:
    df_plot = df_plot.iloc[0:1]
else:
    df_plot = df_plot.loc[:df_plot.index[df_plot.index.get_loc(df_plot[df_plot != df_plot[-1]].index[-1]) + 1]]

In [231]:
fig = px.line(df_plot, markers=True, height=500, width=600)
fig.update_xaxes(tickangle=-70)
fig.update_layout(title=df_plot.name, xaxis_title='Plan Week', yaxis_title='QTY', showlegend=False)

In [ ]:
max_idx = df_plot.duplicated()
max_idx = max_idx[max_idx == False].index.max()
df_plot = df_plot.loc[:max_idx]
df_plot = df_plot.stack().reset_index()

In [88]:
fig = px.line(df_plot)
# fig.update_layout(plot_bgcolor = '#ffffff')
fig.update_xaxes(tickangle=60)

In [77]:
df_plot = monthly_sum(df.groupby(['Series', 'REF']).sum(numeric_only=True).reset_index(), ['Series', 'REF']).loc['13U50N'].stack().reset_index()
df_plot
# df_plot.rename(columns={0:'QTY'}, inplace=True)
# fig = px.line(df_plot, x='REF', y='QTY')
# fig.update_layout(plot_bgcolor='#ffffff')

,REF,level_1,0
0,2021-05-10(W19),2021-04,270.0
1,2021-05-10(W19),2021-05,0.0
2,2021-05-10(W19),2021-06,0.0
3,2021-05-10(W19),2021-07,0.0
4,2021-05-10(W19),2021-08,0.0
...,...,...,...
3625,2023-06-12(W24),2023-08,0.0
3626,2023-06-12(W24),2023-09,0.0
3627,2023-06-12(W24),2023-10,0.0
3628,2023-06-12(W24),2023-11,0.0


In [ ]:
with pd.ExcelWriter(writer, )

In [25]:
df1 = monthly_sum(df.groupby(['Series', 'REF']).sum(numeric_only=True).reset_index(), ['Series', 'REF'])

In [34]:
for sr in df1.index.get_level_values(0).unique():
    print(df1.loc[sr])
    break

                 2021-04  2021-05  2021-06  2021-07  2021-08  2021-09  \
REF                                                                     
2021-05-10(W19)    270.0      0.0      0.0      0.0      0.0      0.0   
2021-05-17(W20)    270.0      0.0      0.0      0.0      0.0      0.0   
2021-05-24(W21)    270.0      0.0      0.0      0.0      0.0      0.0   
2021-05-31(W22)    270.0      0.0      0.0      0.0      0.0      0.0   
2021-06-07(W23)    270.0      0.0      0.0      0.0      0.0      0.0   
...                  ...      ...      ...      ...      ...      ...   
2023-05-15(W20)    270.0      0.0      0.0      0.0      0.0      0.0   
2023-05-22(W21)    270.0      0.0      0.0      0.0      0.0      0.0   
2023-05-29(W22)    270.0      0.0      0.0      0.0      0.0      0.0   
2023-06-05(W23)    270.0      0.0      0.0      0.0      0.0      0.0   
2023-06-12(W24)    270.0      0.0      0.0      0.0      0.0      0.0   

                 2021-10  2021-11  2021-12  2022-0

In [36]:
df1.loc['13U50N']

,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
REF,,,,,,,,,,,,,,,,,,,,,
2021-05-10(W19),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-05-17(W20),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-05-24(W21),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-05-31(W22),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-06-07(W23),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-15(W20),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-22(W21),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023-05-29(W22),270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
sp_exist_model = df1.groupby('Series').sum(numeric_only=True).sum(axis=1)

In [45]:
sp_exist_model

KeyError: '[False, True, False, False, False, False, True, False, False, True, False, True, False, False, False, True] not found in axis'

In [ ]:
df.groupby('Series').sum(numeric_only=True).sum(axis=1)

# GSCP 주별 SP 자료 다운받고 각 주별 변동 이력 시각화 하기

In [138]:
df_plot = monthly_sum(sp_all.groupby(['Ref', 'Updated_at']).sum().loc[pd.MultiIndex.from_frame(sp_all.groupby('Ref')['Updated_at'].max().reset_index())].reset_index(), 'Ref')

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_13628\641350049.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [100]:
df_plot = df_plot.loc[:, df_plot.sum() > 0]

In [102]:
df_plot.to_clipboard()

In [103]:
df_plot

,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11
REF,,,,,,,,,,,,,,,,,,
2022-04-11(W15),0.0,850.0,850.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-18(W16),0.0,850.0,850.0,1200.0,300.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-25(W17),0.0,850.0,850.0,1200.0,400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-02(W18),0.0,850.0,850.0,1200.0,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-05-09(W19),1.0,700.0,1000.0,1200.0,1050.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-22(W21),0.0,1760.0,1080.0,0.0,0.0,1320.0,680.0,1860.0,720.0,220.0,0.0,0.0,0.0,6560.0,0.0,0.0,0.0,0.0
2023-05-29(W22),0.0,1760.0,1080.0,0.0,0.0,1320.0,680.0,1860.0,720.0,220.0,0.0,0.0,0.0,6560.0,0.0,0.0,0.0,0.0
2023-06-05(W23),0.0,1760.0,1080.0,0.0,0.0,1320.0,680.0,1860.0,720.0,220.0,0.0,0.0,0.0,6560.0,0.0,0.0,0.0,0.0


In [113]:
df_plot = acc_sp.groupby('REF').sum().sum(axis=1).reset_index()
df_plot.rename(columns={0:'QTY'}, inplace=True)
fig = px.line(df_plot, x='REF', y='QTY')
fig.update_layout(plot_bgcolor='#ffffff')

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_544\4044175204.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [104]:
fig = px.bar(df_plot.groupby('REF').sum().reset_index(), x='REF', y='QTY', text_auto=True)
fig.update_layout(plot_bgcolor='#ffffff')

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['REF', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11'] but received: QTY

In [87]:
fig = px.line(df_plot.groupby('REF').sum().reset_index(), x='REF', y='QTY')
fig.update_layout(plot_bgcolor='#ffffff')

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_544\801028766.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [106]:
df_plot = df_plot.stack().reset_index()
df_plot.rename(columns={'level_1':'Month', 0:'QTY'}, inplace=True)
fig = px.bar(df_plot, x='REF', y='QTY', color='Month', width=1200, height=800, title = '10A30Q SP History')
fig.update_xaxes(tickangle=60)
fig.update_layout(plot_bgcolor='#ffffff')

In [140]:
max_idx = df_plot.duplicated()
max_idx = max_idx[max_idx == False].index.max()
max_idx

'2023-05-29(W22)'

In [144]:
df_plot = df_plot.loc[:max_idx]
df_plot = df_plot.stack().reset_index()
df_plot.rename(columns={'level_1':'Month', 0:'QTY'}, inplace=True)
fig = px.bar(df_plot, x='Ref', y='QTY', color='Month', width=1200, height=800, title = f'{series} SP History')
fig.update_xaxes(tickangle=60)
fig

### 시각화

In [160]:
update_time = datetime.datetime.now().strftime('%y%m%d%H')
sp_exist_model = df.groupby('Series').sum(numeric_only=True).sum(axis=1)
sp_exist_model = sp_exist_model[sp_exist_model > 0].index
with pd.ExcelWriter(f'D:/Python result/{update_time}_odm_sp_history.xlsx') as writer:
    for num, name in enumerate(sp_exist_model):
        print(f'{num+1}_{name}의 SP 변동 이력 시각화 중 입니다.')
        df_plot = monthly_sum(df[df['Series'] == name].groupby('REF').sum(numeric_only=True).reset_index(), 'REF')
        df_plot.to_excel(writer, sheet_name=name, index=True)
        df_plot = df_plot.loc[:, df_plot.sum() > 0]
        max_idx = df_plot.duplicated()
        max_idx = max_idx[max_idx == False].index.max()
        df_plot = df_plot.loc[:max_idx]
        df_plot = df_plot.stack().reset_index()
        df_plot.rename(columns={'level_1':'Month', 0:'QTY'}, inplace=True)
        fig = px.bar(df_plot, x='REF', y='QTY', color='Month', width=1200, height=800, title = f'{name} SP History')
        fig.update_xaxes(tickangle=70)
        fig.write_image(f'D:/figure/{name}.png')        

1_13U50N의 SP 변동 이력 시각화 중 입니다.
2_13U70P의 SP 변동 이력 시각화 중 입니다.
3_13U70Q의 SP 변동 이력 시각화 중 입니다.
4_14T30Q의 SP 변동 이력 시각화 중 입니다.
5_14T90N의 SP 변동 이력 시각화 중 입니다.
6_14U30P의 SP 변동 이력 시각화 중 입니다.
7_14U30Q의 SP 변동 이력 시각화 중 입니다.
8_14U390의 SP 변동 이력 시각화 중 입니다.
9_17U70N의 SP 변동 이력 시각화 중 입니다.
10_17U70P의 SP 변동 이력 시각화 중 입니다.
11_17U70Q의 SP 변동 이력 시각화 중 입니다.


In [96]:
df = monthly_sum(acc_sp[acc_sp['Series'] == '17U790'].groupby('REF').sum(numeric_only=True).reset_index(), 'REF')
df = df.loc[:, df.sum()>0]

In [6]:
by_series = acc_sp.groupby(['REF', 'Series']).sum(numeric_only=True).sum(axis=1).unstack(0)
by_series['Max Value'] = by_series.max(axis=1)
by_series['Min Value'] = by_series.min(axis=1)
by_series

REF,2018-12-31(W01),2019-01-07(W02),2019-01-14(W03),2019-01-21(W04),2019-01-28(W05),2019-02-04(W06),2019-02-11(W07),2019-02-18(W08),2019-02-25(W09),2019-03-04(W10),...,2023-04-17(W16),2023-04-24(W17),2023-05-01(W18),2023-05-08(W19),2023-05-15(W20),2023-05-22(W21),2023-05-29(W22),2023-06-05(W23),Max Value,Min Value
Series,,,,,,,,,,,,,,,,,,,,,
11T50Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7792.0,7792.0,8332.0,8242.0,8242.0,8232.0,8532.0,8522.0,10752.0,1900.0
13U50N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7350.0,7350.0,7350.0,7350.0,7350.0,7350.0,7350.0,7350.0,7830.0,970.0
13U580,3050.0,3270.0,3270.0,3270.0,4410.0,5215.0,5720.0,5720.0,6130.0,6310.0,...,17328.0,17328.0,17328.0,17328.0,17328.0,17328.0,17328.0,17328.0,17330.0,3050.0
13U70P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27526.0,27526.0,27526.0,27526.0,27526.0,27526.0,27526.0,27526.0,38230.0,7730.0
13U70Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13324.0,13324.0,13324.0,13324.0,13324.0,13324.0,13324.0,13324.0,16371.0,1050.0
14T30Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11680.0,11730.0,11730.0,11730.0,11730.0,11730.0,11730.0,11730.0,38990.0,610.0
14T90N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21899.0,21899.0,21899.0,21899.0,21899.0,21899.0,21899.0,21899.0,22504.0,8455.0
14T90P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27775.0,27775.0,27775.0,27775.0,27775.0,27775.0,27775.0,27775.0,32991.0,10000.0
14T90Q,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8115.0,8115.0,8115.0,8115.0,8115.0,8115.0,8115.0,8115.0,10452.0,200.0


In [7]:
update_time = datetime.datetime.now().strftime('%y%m%d%H')
with pd.ExcelWriter(f'D:/Python result/{update_time}_odm_sp_history.xlsx') as writer:
    acc_sp.to_excel(writer, sheet_name='Raw data', index=False)
    by_series.to_excel(writer, sheet_name='by series', index=True)    

KeyboardInterrupt: 

In [9]:
by_series.to_clipboard()

In [ ]:
from openpyxl.drawing.image import Image
from openpyxl import Workbook
from openpyxl import load_workbook

fn = get_filename()
wb = load_workbook(fn)

for name in sp_exist_model:
    sh = wb[f'{name}']
    add_image = Image(f'D:/figure/{name}.png')
    sh.add_image(add_image, 'A1')

In [166]:
wb.save(fn)

In [29]:
by_series.index

Index(['11T50Q', '13U50N', '13U580', '13U70P', '13U70Q', '14T30Q', '14T90N',
       '14T90P', '14T90Q', '14T90R', '14T990', '14U30P', '14U30Q', '14U380',
       '14U390', '15G880', '15G890', '15U370', '15U40N', '15U40Q', '15U40R',
       '15U470', '15U480', '15U490', '15U50N', '15U50P', '15U50Q', '15U50R',
       '15U590', '15U70N', '15U70P', '15U780', '16T90P', '16T90Q', '16T90R',
       '17U70N', '17U70P', '17U70Q', '17U790', 'CL600'],
      dtype='object', name='Series')

In [46]:
# 누적 SP history 데이터 저장
with open('D:/Data/odm_forecast_history.db', 'wb') as f:
    pickle.dump(acc_sp, f)

In [131]:
acc_sp.groupby(['Series', 'REF']).sum().sum(axis=1).to_clipboard()

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_25376\3974850424.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc_sp.groupby(['Series', 'REF']).sum().sum(axis=1).to_clipboard()


In [127]:
# 모델명을 Series명으로 변환했을 때 변환되지 않은 것이 있는지 확인
acc_sp[~acc_sp['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model1).isin(srt_model1.values())]

,REF,Division,From Site,To Site,Region,Mapping Model.Suffix,Series,2018-12-03(W49),2018-12-10(W50),2018-12-17(W51),...,2023-09-11(W37),2023-09-18(W38),2023-09-25(W39),2023-10-02(W40),2023-10-09(W41),2023-10-16(W42),2023-10-23(W43),2023-10-30(W44),2023-11-06(W45),2023-11-13(W46)
7,2018-12-31(W01),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U380-E.AU1TK,14U380,0.0,220.0,1400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018-12-31(W01),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U380-E.AU2TK,14U380,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,2019-01-07(W02),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U380-E.AU1TK,14U380,0.0,220.0,1400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,2019-01-07(W02),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U380-E.AU2TK,14U380,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,2019-01-14(W03),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U380-E.AU1TK,14U380,0.0,220.0,1400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112737,2023-05-15(W20),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U380-E.AU1TK,14U380,0.0,220.0,1400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112738,2023-05-15(W20),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U380-E.AU2TK,14U380,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113669,2023-05-22(W21),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U30Q-E.A710K,14U30Q,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113670,2023-05-22(W21),PC,OS_PEGATRON_CN_P,EKHQ,KOREA,14U380-E.AU1TK,14U380,0.0,220.0,1400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
# Series 명으로 변환했을 때 변환되지 않은 것의 실제 모델명 확인
acc_sp[~acc_sp['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model1).isin(srt_model1.values())]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).unique()

array(['14U380', '14U30Q'], dtype=object)

In [123]:
srt_model1 = {'15UG50P':'15U50P', '15UD50P':'15U50P', '14TD90P':'14T90P', '16TD90P':'16T90P', '16TG90P':'16T90P',
    '15UD40N':'15U40N', '15UD40Q':'15U40Q', '15UD50N':'15U50N', '15UG50N':'15U50N', '15UD70P':'15U70P',
    '15UG40N':'15U40N', '11TG50Q':'11T50Q', '11TC50Q':'11T50Q', '13UD70P':'13U70P', '13UD70Q':'13U70Q',
    '14UG30P':'14U30P','14UN30P': '14U30P','14UT30P': '14U30P','17UD70P': '17U70P','17UG70P': '17U70P',
    '15GD890':'15G890', '15UD70N':'15U70N', '15UG40Q':'15U40Q', '14TD90Q':'14T90Q', '15UD50Q':'15U50Q',
    '15UG50Q':'15U50Q', '16TD90Q':'16T90Q', 'CL600N':'CL600', 'CL600W':'CL600', '17UD70Q':'17U70Q',
    '14TN30Q':'14T30Q', '14TG30Q':'14T30Q', '14TW30Q':'14T30Q', 'CL600I':'CL600', '16TG90Q':'16T90Q',
    '15UD40R':'15U40R', '16TD90R':'16T90R', '14TD90R':'14T90R', '15UG50R':'15U50R', '15UD50R':'15U50R',
    '15UG40R':'15U40R', '16TG90R':'16T90R', '17UG70Q':'17U70Q', '17UD70N':'17U70N', '17UG70N':'17U70N',
    '13UD50N':'13U50N', '14TD90N':'14T90N', '14UG390':'14U390', '13UD580':'13U580', '15UD370':'15U370', '15UD780':'15U780',
    '15GD880':'15G880', '15UB470':'15U470', '15UD480':'15U480', '15UG470':'15U470', '15UG480':'15U480', '15UD590':'15U590',
    '14TD990':'14T990', '15UD490':'15U490', '17UD790':'17U790', '15UG590':'15U590', '14TG90N':'14T90N'}